<a href="https://colab.research.google.com/github/carolynw898/STAT946Proj/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch

n_embd = 512
timesteps = 1000
batch_size = 1
learning_rate = 1e-4
num_epochs = 10
blockSize = 32
testBlockSize = 400
numVars = 1
numYs = 1
numPoints = 250
target = 'Skeleton'
const_range = [-2.1, 2.1]
trainRange = [-3.0, 3.0]
decimals = 8
addVars = False
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from torch.utils.data import DataLoader
import numpy as np
import glob
from utils import processDataFiles, CharDataset, tokenize_equation
import random
import json

files = glob.glob("/content/drive/MyDrive/Colab Notebooks/STAT946_proj/data/1_var_train.json")
text = processDataFiles(files)
text = text.split('\n') # convert the raw text to a set of examples
# skeletons = []
skeletons = [json.loads(item)['Skeleton'] for item in text if item.strip()]
all_tokens = set()
for eq in skeletons:
    all_tokens.update(tokenize_equation(eq))
integers = {'0', '1', '2', '3', '4', '5', '6', '7', '8', '9'}
all_tokens.update(integers)  # add all integers to the token set
tokens = sorted(list(all_tokens) + ['_', 'T', '<', '>', ':'])  # special tokens
trainText = text[:-1] if len(text[-1]) == 0 else text
random.shuffle(trainText) # shuffle the dataset, it's important specailly for the combined number of variables experiment
train_dataset = CharDataset(trainText, blockSize, tokens=tokens, numVars=numVars,
                        numYs=numYs, numPoints=numPoints, target=target, addVars=addVars,
                        const_range=const_range, xRange=trainRange, decimals=decimals)

idx = np.random.randint(train_dataset.__len__())
inputs, outputs, points, variables = train_dataset.__getitem__(idx)
inputs = ''.join([train_dataset.itos[int(i)] for i in inputs])
outputs = ''.join([train_dataset.itos[int(i)] for i in outputs])
print('id:{}\noutputs:{}\nvariables:{}'.format(idx,outputs,variables))

data has 498795 examples, 27 unique.
id:447738
outputs:C*log(C*x1)+C>_____________________
variables:1


In [22]:
files = glob.glob("/content/drive/MyDrive/Colab Notebooks/STAT946_proj/data/1_var_test.json")
textTest = processDataFiles([files[0]])
textTest = textTest.split('\n') # convert the raw text to a set of examples
test_dataset = CharDataset(textTest, blockSize, tokens=tokens, numVars=numVars,
                        numYs=numYs, numPoints=numPoints, target=target, addVars=addVars,
                        const_range=const_range, xRange=trainRange, decimals=decimals)

# print a random sample
idx = np.random.randint(test_dataset.__len__())
inputs, outputs, points, variables = test_dataset.__getitem__(idx)
print(points.min(), points.max())
inputs = ''.join([train_dataset.itos[int(i)] for i in inputs])
outputs = ''.join([train_dataset.itos[int(i)] for i in outputs])
print('id:{}\noutputs:{}\nvariables:{}'.format(idx,outputs,variables))

test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        pin_memory=True,
        shuffle=False,
        num_workers=2
    )

data has 967 examples, 27 unique.
tensor(-7.8646) tensor(2.9267)
id:34
outputs:C*log(C*x1)+C>_____________________
variables:1


In [ ]:
import torch
import json
from tqdm import tqdm
from utils import sample_and_select_best

@torch.no_grad()
def test_model(model, test_loader, textTest, train_dataset, device, ddim_step=20, numTests=10):
    results = {"target": [], "target_skeleton": [], "predicted_skeleton": [], "predicted": [], "error": []}
    for batch_idx, (_, tokens, points, variables) in tqdm(
        enumerate(test_loader),
        total=len(test_loader),
        desc="Processing Test Batches",
        unit="batch"
    ):
        points = points.to(device)
        tokens = tokens.to(device)
        variables = variables.to(device)
        B = points.shape[0]
        print(f'points.shape: {points.shape}')

        for i in tqdm(
            range(B),
            total=B,
            desc=f"Batch {batch_idx+1}/{len(test_loader)} Samples",
            unit="sample",
            leave=False  
        ):
            t = json.loads(textTest[batch_idx * test_loader.batch_size + i])
            eq = t["EQ"]

            eq_skeleton = ''.join([train_dataset.itos[int(k)] for k in tokens[i]])
            eq_skeleton = eq_skeleton.strip(train_dataset.paddingToken).split('>')[0].strip('<').strip(">")

            points_i = points[i:i+1]
            variables_i = variables[i:i+1]
            predicted_skeleton, predicted, err = sample_and_select_best(
                model, points_i, variables_i, train_dataset, eq, t, numTests, ddim_step
            )

            #print(f'eq: {eq}')
            #print(f'eq_skeleton: {eq_skeleton}')
            #print(f'predicted_skeleton: {predicted_skeleton}')
            #print(f'predicted: {predicted}')
            #print(f'err: {err}\n')

            results["target"].append(eq)
            results["target_skeleton"].append(eq_skeleton)
            results["predicted_skeleton"].append(predicted_skeleton)
            results["predicted"].append(predicted)
            results["error"].append(err)

    return results

In [ ]:
from utils import plot_and_save_results

def run_and_plot(model, fName, test_loader, textTest, train_dataset, device, pconf=None, ddim_step=20, numTests=10):
    """Run test_model and plot results using utility functions.
    Args:
        model: SymbolicGaussianDiffusion model
        test_loader: DataLoader for test data
        textTest: list of test data
        train_dataset: dataset with itos mapping
        device: torch device
        pconf: PointNetConfig object
    """
    results = test_model(model, test_loader, textTest, train_dataset, device, ddim_step, numTests)

    modelKey = 'SymbolicDiffusion'
    resultDict = {
        fName: {
            modelKey: {
                'err': results['error'],
                'trg': results['target'],
                'prd': results['predicted']
            }
        }
    }

    titleTemplate = "{} equations of {} variables - Benchmark"
    plot_and_save_results(resultDict, fName, pconf, titleTemplate, textTest, modelKey=modelKey)

In [ ]:
from models import SymbolicGaussianDiffusion, PointNetConfig

pconfig = PointNetConfig(
    embeddingSize=n_embd,
    numberofPoints=250,
    numberofVars=1,
    numberofYs=1,
)
model = SymbolicGaussianDiffusion(
    tnet_config=pconfig,
    vocab_size=train_dataset.vocab_size,
    max_seq_len=32,
    padding_idx=test_dataset.paddingID,
    max_num_vars=9,
    n_layer=4,
    n_head=4,
    n_embd=n_embd,
    timesteps=timesteps,
    beta_start=0.0001,
    beta_end=0.02,
).to(device)

model_path = "/content/drive/MyDrive/Colab Notebooks/STAT946_proj/models/best_model_116.pth"
model.load_state_dict(torch.load(model_path, map_location=device, weights_only=True))
model.eval()

fName = "116.txt"
test_results = run_and_plot(model, fName, test_loader, textTest, train_dataset, device, pconfig, ddim_step=5, numTests=20)